In [248]:
from konlpy.tag import Twitter
from gensim.models import Word2Vec
import csv
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
import matplotlib

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

import csv
import logging
from konlpy import jvm
logger = logging.getLogger(__name__)
jvm.init_jvm()



In [249]:
data_set.head()

,news,label
0,대구에서 518명단 공개 주장한 김진태 이해찬은 왜 들어갔나,3
1,조폭 50대 부동산업자 살해 양주서 시신 발견,4
2,김환기 우주 132억원 낙찰 짜릿하면서도 씁쓸한 뒷맛,1
3,2차 정상회담 D7김정은은 왜 베트남을 택했나,3
4,민주당 입당 고민정 출마지 당에 맡겼다광진을동작을 거론,3


In [250]:
data_set = pd.read_excel("data.xlsx")
data_set = data_set.iloc[:,1:]
data_set.rename({0:'news'},axis=1, inplace= True)
data_label = {1:'culture',2:'global',3:'politic',4:'society',5:'economy'}




In [251]:
for i in range(len(data_set)) :
    data_set['news'][i] = re.sub('[^a-zA-Z0-9 ㄱ-ㅣ가-힣]', '', data_set['news'][i])
    


C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [252]:
data_set = data_set.sample(frac=1).reset_index(drop=True)

In [253]:
data_set.head()
len(data_set)
data_set2 = data_set.iloc[:20000,:]
len(data_set2)

20000

In [254]:
def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun2(text):
    tokenizer = Hannanum()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun3(text):
    tokenizer = Okt()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_morphs3(text):
    tokenizer = Okt()
    morphs = tokenizer.morphs(text)
    return [n for n in morphs]

def get_noun4(text):
    tokenizer = Komoran()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]


In [255]:

CV = CountVectorizer(tokenizer = get_noun3)
CV2 = CountVectorizer(tokenizer = get_morphs3)


In [256]:
CV = CountVectorizer(tokenizer = get_noun3)

In [257]:
#train_set, test_set = train_test_split(data_set, test_size = 0.3, random_state=42)
train_set, test_set = data_set2.iloc[:15000,:].reset_index(drop=True), data_set2.iloc[15000:,:].reset_index(drop=True)
print(len(train_set))
print(len(test_set))


15000
5000


In [258]:
print(len(test_set['news']))
print(len(test_set['label']))

5000
5000


In [259]:
test_set['news'][0]

'진영 장관 아산 주민 달래러 갔다 계란 봉변'

In [260]:
train_set['news'][0]
test_set

,news,label
0,진영 장관 아산 주민 달래러 갔다 계란 봉변,3
1,속보연체 채무자 180만명에 채무조정 협상 요청권 부여 추진,5
2,하와이 스카이다이빙 비행기 추락11명 사망종합,2
3,공중보건비상사태 선포2주내 중국 방문한 외국인 입국,2
4,민경욱 김정숙 여사 황대표 악수 생략 시간여유 없었다종,3
...,...,...
4995,뉴질랜드 여행 중 실종된 20대 남성 숨진 채 발견,2
4996,위장약인 줄 알았는데 발모제 늑대인간 된 16명 아기들,2
4997,임블리 호박즙 곰팡이 죄송 환불하겠다,5
4998,탈원전에월성 234호기 스톱 초유의 사태 온다,5


In [262]:
len(train_set)
len(data_set)
print(type(train_set))
type(data_set)

<class 'pandas.core.frame.DataFrame'>


pandas.core.frame.DataFrame

In [263]:
label_set = train_set['label']

In [264]:
from konlpy.tag import Twitter
from konlpy.tag import Kkma
twitter = Twitter()
token = []
embeddingmodel = []
for i in range(len(train_set)):
    label = train_set['label']
    sentence = twitter.pos(train_set['news'][i], norm=True, stem=True)
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0])
        temp.append(sentence[k][0] + '/' + sentence[k][1])
    all_temp.append(temp)
    embeddingmodel.append(temp_embedding)
    
    all_temp.append(label[i])
    token.append(all_temp)
    
embeddingmodel = []
for i in range(len(label_set)):
    temp_embeddingmodel = []
    for k in range(len(token[i][0])):
        temp_embeddingmodel.append(token[i][0][k])
    embeddingmodel.append(temp_embeddingmodel)
embedding  = Word2Vec(embeddingmodel, size = 300, window =5, min_count = 10,
                      iter = 5, sg=1)

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [265]:
len(test_set['label'])
len(test_set['news'])
test_set['news'].shape
#token


(5000,)

In [266]:
def One_hot(data):
       
    index_dict = {value:index for index,value in enumerate(set(data))}
    result = []

    for value in data:

        one_hot = np.zeros(len(index_dict))
        index = index_dict[value]
        one_hot[index] = 1
        result.append(one_hot)

    return result


def Convert2Vec(model_name, doc): # Convert corpus into vectors
    #train_X_ = W2V.Convert2Vec("Word2Vec_csv_article.embedding",train_X)
    word_vec = []
    model = model_name
    for sent in doc:
        sub = []
        for word in sent:
            if word in model.wv.vocab:
                sub.append(model.wv[word]) # Word Vector Input
            else:
                sub.append(np.random.uniform(-0.25,0.25,300)) # used for OOV words
        word_vec.append(sub)

    return word_vec





In [267]:
class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
            
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
        
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,dtype=tf.float32,
                                                                            inputs = X, sequence_length = seq_len)
        ## concat fw, bw final states
        outputs = tf.concat([states[0][1], states[1][1]], axis=1)
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # Adam Optimizer
            
        return loss, optimizer
    
    def graph_build(self, avg_loss, avg_acc):
        
        tf.summary.scalar('Loss', avg_loss)
        tf.summary.scalar('Accuracy', avg_acc)
        merged = tf.summary.merge_all()
        return merged

In [268]:
def Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
    zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
    for i in range(Batch_size):
        zero_pad[i, :np.shape(train_batch_X[i])[0], :np.shape(train_batch_X[i])[1] ] = train_batch_X[i]
    return zero_pad

In [269]:
tokens = np.array(token)
print(tokens.shape)
train_x = tokens[:,0]
print(train_x.shape)
train_y = tokens[:,1]
print(train_y.shape)

train_y = One_hot(train_y)
train_x = Convert2Vec(embedding, train_x)

(15000, 2)
(15000,)
(15000,)


In [270]:
Batch_size = 32
Total_size = len(train_x)
Vector_size = 300
seq_length = [len(x) for x in train_x]
Maxseq_length = max(seq_length)
learning_rate = 0.001
lstm_units = 128
num_class = 5
training_epochs = 20
keep_prob = 0.75

X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size])
Y = tf.placeholder(tf.float32, shape = [None, num_class])
seq_len = tf.placeholder(tf.int32 , shape = [None])

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)
with tf.variable_scope('loss', reuse = tf.AUTO_REUSE) :
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

total_batch = int(Total_size / Batch_size)

print("Start training!")

#modelName = "BiLSTM.model"
saver = tf.train.Saver()

Start training!


In [271]:
import time
import os
with tf.Session() as sess:

    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter('Bidirectional_LSTM', sess.graph)
    i = 0
    for epoch in range(training_epochs):

        avg_acc, avg_loss = 0. , 0.
        for step in range(total_batch):

            train_batch_X = train_x[step*Batch_size : step*Batch_size+Batch_size]
            
            train_batch_Y = train_y[step*Batch_size : step*Batch_size+Batch_size]
            batch_seq_length = seq_length[step*Batch_size : step*Batch_size+Batch_size]
            train_batch_X = Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
            
            sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            # Compute average loss
            loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            avg_loss += loss_ / total_batch
            
            acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            avg_acc += acc / total_batch
        print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))
        print("Test Accuracy : ", sess.run(accuracy, feed_dict = {X:test_set['news'], Y: test_set['label']}))
        summary = sess.run(BiLSTM.graph_build(avg_loss, avg_acc))       
        train_writer.add_summary(summary, i)
        i += 1

    duration = time.time() - start_time
    minute = int(duration / 60)
    second = int(duration) % 60
    print("%dminutes %dseconds" % (minute,second))
    save_path = saver.save(sess, os.getcwd())

    train_writer.close()
    print('save_path',save_path)

epoch : 01 step : 0468 loss = 1.261155 accuracy= 0.406250


ValueError: could not convert string to float: '진영 장관 아산 주민 달래러 갔다 계란 봉변'

In [ ]:
def Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
    zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
    for i in range(Batch_size):
        zero_pad[i, :np.shape(train_batch_X[i])[0], :np.shape(train_batch_X[i])[1] ] = train_batch_X[i]
    return zero_pad



In [ ]:
train_batch_X = Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)


In [213]:
zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))

for i in range(Batch_size):
    zero_pad[i, :np.shape(train_batch_X[i])[0], :np.shape(train_batch_X[i])[1] ] = train_batch_X[i]
zero_pad[:100,:,:]

IndexError: tuple index out of range